In [1]:
import pandas as pd
import openpyxl
from collections import Counter
import xlrd
import xlwt
import os
import time
from openpyxl import Workbook
import numpy as np
from itertools import islice

countiter = 0

#from openpyxl.utils.dataframe import dataframe_to_row
#from openpyxl.cell import get_column_letter, column_index_from_srting
start_time = time.time()

indir = 'C:/Users/d.savchenko/python/true_buying/files/'   #работающий формат пути!!!
indir_buying = 'C:/Users/d.savchenko/python/true_buying/'

df_buying = pd.read_excel(indir_buying+"Buying_Variable.xlsx")
#print(df_buying)

#df_final = pd.DataFrame(index=index, columns=columns)
flag0 = False

for root, dirs, filenames in os.walk(indir):
    #print("ya rabotayu")
   
    for f in filenames:
        print("--- %s seconds ---" % (time.time() - start_time))

for root, dirs, filenames in os.walk(indir):
    #print("ya rabotayu")
   
    for f in filenames: 
        if flag0 == False:    
            flag0 = True
            print("---до excel %s seconds ---" % (time.time() - start_time))
            wb = openpyxl.load_workbook(indir+f)
            print("---после excel %s seconds ---" % (time.time() - start_time))
            sheet = wb.active
            print("--- %s seconds ---" % (time.time() - start_time))



            if sheet.cell(row=2, column=1).value is not None:
                print("Проверьте выгрузку Вторая строчка не содержит пустых значений!!!") 

            flag = False
            col_end_pokazat=0
            qu_TA = 1
            for j in range(1, sheet.max_column):
                cell_val = sheet.cell(row=1, column=j).value
                
                if (cell_val is None) and flag == False:
                    flag = True
                    col_end_pokazat = j - 2

                if flag == True:
                    if cell_val is not None:
                        qu_TA = qu_TA + 1
                    else:
                        qu_TA = qu_TA


                #print(cell_val)

            qu_perem = int((sheet.max_column - col_end_pokazat)/qu_TA)
            #k = sheet['A5'].value
            print('количество колонок с показателями:',  col_end_pokazat)
            print('количество TA:', qu_TA)
            print('количество переменных:', qu_perem)

            val_cell_TA = " "

            for j in range(col_end_pokazat+1, sheet.max_column+1):
                cell_val = sheet.cell(row=1, column=j).value
                if cell_val is not None:
                    val_cell_TA = cell_val
                    sheet.cell(row=1, column=j, value= val_cell_TA + sheet.cell(row=2, column=j).value)    
                else:
                    sheet.cell(row=1, column=j, value= val_cell_TA + sheet.cell(row=2, column=j).value)
            for j in range(sheet.max_column+1, sheet.max_column+1+qu_perem):
                sheet.cell(row=1, column=j, value= "buying " + sheet.cell(row=2, column=j-qu_perem).value)
#--------------------------добавляются баинговые строки------------------------------------------------------------

            data = sheet.values
            cols = next(data)[1:]
            wb.close()
            print("---до считывания %s seconds ---" % (time.time() - start_time))
            
            print("строим кортеж--- %s seconds ---" % (time.time() - start_time))
            d = {}
            channel = []
            target = []
            for j in range(1, df_buying.shape[0]):
                d[df_buying["CHANNEL"][j]] = df_buying["TARGET"][j] 
            #print(d)
            #print("adsdsd",d["2019NEWS ONE"])
            print("закончил кортеж, начал в цикле забивать целевые--- %s seconds ---" % (time.time() - start_time))
         
            
            
        df = pd.read_excel(indir+f)
        print("---после считывания пандасом %s seconds ---" % (time.time() - start_time))

        #print(df)
        del df[df.columns[0]]

        print("новые стообики--- %s seconds ---" % (time.time() - start_time))
        df["new"] = "-"
        df["new2"] = "-"
        df["new3"] = "-"
        df["new4"] = "-"

        df.columns = cols

        df = df.drop(df.index[[0]])

        print("--- %s seconds ---" % (time.time() - start_time))
        #df2 = df
        #df = df.astype({"Year": int})
        #df = df.astype({"Year": str})
        #df2 = df2.astype({"Year": int})
        #df2 = df2.astype({"Year": str})

        #df2 = df2["Year"]+df2["Channel"]

#------------------------------------------------------------------------------------------------------------
       # print(df_buying[df_buying[df2["YearChannel"]]][0])
#---------------------------------------------------------------------------------------------------------------

               
#разбиение на 20--------------------------------------------------------------------------------------------------
        
        df_split = ["df_1","df_2","df_3","df_4","df_5","df_6","df_7","df_8","df_9","df_10",
                    "df_11","df_12","df_13","df_14","df_15","df_16","df_17","df_18","df_19","df_20"]
        filerange = df.shape[0]//len(df_split)
        dfsh = df.shape[0]
        print(filerange)
        
        for c in range(len(df_split)):
            
            if c==len(df_split)-1:
                df_split[c] = df[:dfsh]
            else:
                df_split[c] = df[:filerange]
                df = df.drop(df.index[:filerange])
                if c==0:
                    df_final = df_split[0]
        
            df2 = df_split[c]
            df2 = df2.astype({"Year": int})
            df2 = df2.astype({"Year": str})
            #print(df2["Year"])
            df2 = df2["Year"]+df2["Channel"]
            
            if c==len(df_split)-1:
                start = c*filerange+1
                finish =dfsh+1
            else:
                start = c*filerange+1
                finish =(c+1)*filerange+1
            for i in range(start,finish):  #строки
                #print("начал в цикле забивать целевые--- %s seconds ---" % (time.time() - start_time))
                #print(i)
                df_split[c].loc[i, df_split[c].columns[df_split[c].shape[1]-1]] = d[df2[i]] + "EqPrice"
                df_split[c].loc[i, df_split[c].columns[df_split[c].shape[1]-2]] = d[df2[i]] + "EqGRP%"
                df_split[c].loc[i, df_split[c].columns[df_split[c].shape[1]-3]] = d[df2[i]] + "WGRP%"
                df_split[c].loc[i, df_split[c].columns[df_split[c].shape[1]-4]] = d[df2[i]] + "grp%"
            #print(df_1)
            print(" закончил целевые --- %s seconds ---" % (time.time() - start_time))
            
            df_split[c][df_split[c].columns[df_split[c].shape[1]-1]] = df_split[c][df_split[c][df_split[c].columns[df_split[c].shape[1]-1]]] 
            #print(" закончил целевые закончил с 1ім столбцом --- %s seconds ---" % (time.time() - start_time))
            df_split[c][df_split[c].columns[df_split[c].shape[1]-2]] = df_split[c][df_split[c][df_split[c].columns[df_split[c].shape[1]-2]]]
           # print(" закончил целевые закончил с 2ім столбцом --- %s seconds ---" % (time.time() - start_time))
            df_split[c][df_split[c].columns[df_split[c].shape[1]-3]] = df_split[c][df_split[c][df_split[c].columns[df_split[c].shape[1]-3]]]
            #print(" закончил целевые закончил с 3ім столбцом --- %s seconds ---" % (time.time() - start_time))
            df_split[c][df_split[c].columns[df_split[c].shape[1]-4]] = df_split[c][df_split[c][df_split[c].columns[df_split[c].shape[1]-4]]]

            print(" переприсвоило, можно оптимизировать --- %s seconds ---" % (time.time() - start_time))

    
#--------------------------------------------------------------------------------------------------
        
            if c > 0:
                df_final = pd.concat([df_final, df_split[c]], sort=False, ignore_index=True)
        #df_final = pd.concat([df_final, df], sort=False, ignore_index=True)  #сбор в 1 файл

        df_final.to_excel(indir+f+' with_buying.xlsx')
        
        
print("--- %s seconds ---" % (time.time() - start_time))
#df_final.to_excel(indir+'Final.xlsx')
print(countiter)
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.04799675941467285 seconds ---
--- 0.04799675941467285 seconds ---
--- 0.04799675941467285 seconds ---
--- 0.04799675941467285 seconds ---
--- 0.04799675941467285 seconds ---
---до excel 0.04799675941467285 seconds ---
---после excel 79.03900933265686 seconds ---
--- 79.03900933265686 seconds ---
количество колонок с показателями: 22
количество TA: 49
количество переменных: 4
---до считывания 81.630042552948 seconds ---
строим кортеж--- 81.630042552948 seconds ---
закончил кортеж, начал в цикле забивать целевые--- 81.630042552948 seconds ---
---после считывания пандасом 132.09868621826172 seconds ---
новые стообики--- 132.09868621826172 seconds ---
--- 132.29264616966248 seconds ---
1168
 закончил целевые --- 135.6072723865509 seconds ---
 переприсвоило, можно оптимизировать --- 135.72311520576477 seconds ---
 закончил целевые --- 138.84609293937683 seconds ---
 переприсвоило, можно оптимизировать --- 138.96191096305847 seconds ---
 закончил целевые --- 142.12289428710938 seconds 

 закончил целевые --- 575.646452665329 seconds ---
 переприсвоило, можно оптимизировать --- 575.8248691558838 seconds ---
---после считывания пандасом 719.4488482475281 seconds ---
новые стообики--- 719.4488482475281 seconds ---
--- 719.6650364398956 seconds ---
1392
 закончил целевые --- 723.707414150238 seconds ---
 переприсвоило, можно оптимизировать --- 723.907879114151 seconds ---
 закончил целевые --- 727.3700172901154 seconds ---
 переприсвоило, можно оптимизировать --- 727.5549654960632 seconds ---
 закончил целевые --- 731.0491116046906 seconds ---
 переприсвоило, можно оптимизировать --- 731.2430789470673 seconds ---
 закончил целевые --- 734.7593379020691 seconds ---
 переприсвоило, можно оптимизировать --- 734.9377825260162 seconds ---
 закончил целевые --- 738.4625561237335 seconds ---
 переприсвоило, можно оптимизировать --- 738.6474606990814 seconds ---
 закончил целевые --- 742.1724281311035 seconds ---
 переприсвоило, можно оптимизировать --- 742.3573753833771 seconds 